# A Smith Chart Demo

Written by Michael Lamoureux, September 2020.
With guidance from Matt Yedlin and Sam Ng on how to use a Smith chart.

First and foremost, run the following cell to install `plotly`. The installation could take some time, wait for the cell to complete and for the output to be shown. You might also have to restart the Jupyter Notebook server after you have installed it.

In [ ]:
pip install plotly

---

Run the two cells below, or hit Run All under the "Kernel" menu.

There are two sliders where you can input the normalized impedance values (real and imaginary parts). Note that there can be a slight delay after you have changed the values for the plot to catch up.

The code does all the plotting of various points, curves, and lines, as per Sam's instructions. 

Use the check boxes to see the various curves. (A pedogogical tool to help the students see the steps.)

**Make sure `smith_chart.png` is in the same directory as this notebook.**

In [ ]:
%reset -f

import plotly.graph_objects as go
import plotly.offline as plo
import numpy as np
from ipywidgets import HTML, HBox, VBox, BoundedFloatText, Checkbox, interact, interactive


In [ ]:
# define some colors because I am lazy
Red = '#ff0000'
Green = '#00ff00'
Blue = '#0000ff'
Orange = '#ff9600'

plo.init_notebook_mode(connected=False)

# define the mobius transformation, in a convenient form for plotting
def mob(x,y):
    z = (x+1j*y -1)/(x+1j*y +1)
    return [np.real(z),np.imag(z),np.absolute(z)]

# The initial impedance (from Sam's example)
X0,Y0 = 0.8,-1.2 

# set up the figure, with the bounding box. In red
fig = go.FigureWidget([go.Scatter(x=[-1, 1,1,-1,-1], y=[-1,-1,1,1,-1],name='Box')])
theBox = fig.data[0]
theBox.marker.color = Red
fig.layout.hovermode = 'closest'


# Add the data point (impedance)
fig.add_trace(
    go.Scatter(x=[mob(X0,Y0)[0]], y=[mob(X0,Y0)[1]], mode='markers', name='The Point', 
               hovertemplate=f'({X0:.2f}, j{Y0:.2f})')
)
thePoint = fig.data[1]
thePoint.marker.color = Red
thePoint.marker.size = [20]

# Add the constant resistance circle
x = X0
y = np.geomspace(0.01,50,100) # geometric spacing looks better
y = np.append(-np.flip(y),y) # include the negative values
fig.add_trace(
    go.Scatter(
        visible=False,
        line=dict(color=Green, width=3),
        name='Const R',
        x=mob(x,y)[0],
        y=mob(x,y)[1]
        )
    )
theCurve_R = fig.data[2]

# Add the constant impedance curve
x = np.geomspace(0.01,50,100) # geometric spacing looks better
y = Y0
fig.add_trace(
    go.Scatter(
        visible=False,
        line=dict(color=Orange, width=3),
        name='Const X',
        x=mob(x,y)[0],
        y=mob(x,y)[1]
        )
    )
theCurve_X = fig.data[3]

# Add the circle curve
t = np.linspace(0,2*np.pi,100) 
r = np.sqrt(mob(X0,Y0)[0]**2 + mob(X0,Y0)[1]**2)

fig.add_trace(
    go.Scatter(
        visible=False,
        line=dict(color=Blue, width=3),
        name='Circle Gamma',
        x=r*np.cos(t),
        y=r*np.sin(t)
        )
    )
theCircle = fig.data[4]

# add the 1st straight line
r = np.sqrt(mob(X0,Y0)[0]**2 + mob(X0,Y0)[1]**2)
y1 = np.sqrt(4 + 25*r**2)-2
y1 = np.sqrt(y1) # a second square root as we are solving a quartic
x = mob(1,y1)[0]
y = mob(1,y1)[1]
a = max(np.abs(x),np.abs(y))
fig.add_trace(
    go.Scatter(
        visible=False,
        line=dict(color=Blue, width=3),
        name='Line-1',
        x=[0,x,x/a],
        y=[0,y,y/a]
        )
    )
theLine_1 = fig.data[5]

# add the 2nd straight lines
# use same y1 value as above
x = mob(1,-y1)[0]
y = mob(1,-y1)[1]
a = max(np.abs(x),np.abs(y))
fig.add_trace(
    go.Scatter(
        visible=False,
        line=dict(color=Blue, width=3),
        name='Line-2',
        x=[0,x,x/a],
        y=[0,y,y/a]
        )
    )
theLine_2 = fig.data[6]


## update all the curves when X0,Y0 change
def update_curves(X,Y):
    X0 = min(max(X,0),50)
    Y0 = min(max(Y,-50),50)
    # update the point
    thePoint.x = [mob(X0,Y0)[0]]
    thePoint.y = [mob(X0,Y0)[1]]
    thePoint.hovertemplate=f'({X:.2f}, j{Y:.2f})'
    # update the R curve
    x = X0
    y = np.geomspace(0.01,50,100) 
    y = np.append(-np.flip(y),y)
    theCurve_R.x=mob(x,y)[0]
    theCurve_R.y=mob(x,y)[1]
    theCurve_R.hovertemplate=f'Impedance circle for R={X:.2f}'
    # update the C curve
    x = np.geomspace(0.01,50,100) 
    y = Y0
    theCurve_X.x=mob(x,y)[0]
    theCurve_X.y=mob(x,y)[1]
    theCurve_X.hovertemplate=f'Reactance circle for X={Y:.2f}'
    # update the circle
    t = np.linspace(0,2*np.pi,100) 
    r = np.sqrt(mob(X0,Y0)[0]**2 + mob(X0,Y0)[1]**2)
    theCircle.x = r*np.cos(t)
    theCircle.y = r*np.sin(t)
    theCircle.hovertemplate=f'Wave impedances with |Gamma|={mob(X0,Y0)[2]:.2f}'
    # update the lines
    #r = np.sqrt(mob(X0,Y0)[0]**2 + mob(X0,Y0)[1]**2)
    #y1 = np.sqrt(4 + 25*r**2)-2
    #y1 = np.sqrt(y1) # a second square root as we are solving a quartic
    #x = mob(1,y1)[0]
    #y = mob(1,y1)[1]
    ##a = max(np.abs(x),np.abs(y))
    #theLine_1.x =[0,x,x/a]
    #theLine_1.y = [0,y,y/a]
    #x = mob(1,-y1)[0]
    #y = mob(1,-y1)[1]
    #a = max(np.abs(x),np.abs(y))
    #theLine_2.x =[0,x,x/a]
    #theLine_2.y = [0,y,y/a]


    

# Add image in the background
fig.add_layout_image(
        dict(
            source="smith_chart.png",
            xref="x",
            yref="y",
            x=-1.15,
            y=1.15,
            sizex=2.3,
            sizey=2.3,
            sizing="fill",
            opacity=1.0,
            layer="below")
)

fig.update_layout(
    width = 1000,
    height = 1000,
    title = "Smith Chart Display",
    yaxis = dict(
      scaleanchor = "x",
      scaleratio = 1,
    )
)



@interact(Z_real = (0.1,10.0,.1),Z_imag=(-20,20,.1))
def g1(Z_real=X0, Z_imag=Y0):
    update_curves(Z_real,Z_imag)
    return

#@interact(Show_Point=True,Show_Const_R=True,Show_Const_C=True,Show_Circle_1=True,Show_Line_1=False,Show_Line_2=False)
#def g2(Show_Point,Show_Const_R,Show_Const_C,Show_Circle_1,Show_Line_1,Show_Line_2):
@interact(Show_Point=True,Show_Const_R=True,Show_Const_X=True,Show_Circle_Gamma=True)
def g2(Show_Point,Show_Const_R,Show_Const_X,Show_Circle_Gamma):
    thePoint.visible = Show_Point
    theCurve_R.visible = Show_Const_R
    theCurve_X.visible = Show_Const_X
    theCircle.visible = Show_Circle_Gamma
    #theLine_1.visible = Show_Line_1
    #theLine_2.visible = Show_Line_2
    return

fig